<a href="https://colab.research.google.com/github/JoseJardimPV/JoseJardimPV/blob/main/Tratamento_datasets_Importa%C3%A7%C3%A3o_Com%C3%A9rcio_Brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação e importação das bibliotecas

In [ ]:
# INSTALAÇÃO DA BIBLIOTECA GOOGLE CLOUD STORAGE SYSTEM
# (para fazer o upload dos datafiles no bucket da GCP, e também para acessá-los posteriormente)
!pip install gcsfs

In [ ]:
# para acessar a GCP
from google.cloud import storage
from google.oauth2 import service_account 
import os

#pandas
import pandas as pd
import pandas_gbq

# Preparação das bases referentes à importação - José

## Dataset de importação com base nos municípios

In [ ]:
# CONFIGURAÇÃO DA CHAVE DE SEGURANÇA PARA CONEXÃO COM A GCP
serviceAccount = '/content/chave_bucket_JSON.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# ACESSO DA BUCKET
client = storage.Client()

# VARIÁVEL PARA RECEBER O NOME DA BUCKET 
bucket = client.get_bucket('bc26-datasets-comex')

# SELEÇÃO DO ARQUIVO NA BUCKET
bucket.blob('IMP_COMPLETA (MUN).csv')

# VARIÁVEL PARA RECEBER O CAMINHO DO ARQUIVO
path = 'gs://bc26-datasets-comex/IMP_COMPLETA (MUN).csv'

In [ ]:
#CRIAÇÃO DO DF A PARTIR DO CSV (BASE DE DADOS DE IMPORTAÇÃO, BASEADA NOS MUNICÍPIOS BRASILEIROS)
df1 = pd.read_csv(path, sep=";", encoding='ISO-8859-1')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


O primeiro passo para preparar essa base de dados é fazer a seleção do período que será trabalhado nas análises (2012 a 2022).
No entanto foi verificado que não seria possível filtrar o DF pela coluna que faz referência ao ano ("CO_ANO") por comparação matemática, isso porque o formato da coluna não estava compatível com essa filtragem. 

In [ ]:
# pode-se conferir que a coluna "CO_ANO" está como objeto. 
# Portanto deve haver alguma inconsitência, ja que os seus dados deveriam ser apenas números
df1.dtypes

CO_ANO        object
CO_MES         int64
SH4            int64
CO_PAIS        int64
SG_UF_MUN     object
CO_MUN         int64
KG_LIQUIDO     int64
VL_FOB         int64
dtype: object

In [ ]:
# identificação das inconsistências da coluna ("2°07" e "2°06 " não são números)
df1['CO_ANO'].value_counts()

2021    1556169
2019    1527082
2018    1480916
2014    1447599
2022    1446778
2017    1437330
2020    1418639
2013    1417266
2015    1388385
2012    1351537
2016    1335947
2011    1306943
2010    1214230
2008    1056351
2009    1022007
2007     905183
2006     808079
2005     803166
2004     750464
1998     687010
2001     686684
2003     676529
2000     674772
2002     666259
1997     644522
1999     615894
2007      65535
2006      65535
2°07          1
2°06          1
Name: CO_ANO, dtype: int64

In [ ]:
# como não serão utilizados os dados de antes de 2012, é possível apenas dropar as linhas 
# que possuem os erros "2°07" e "2°06" para depois transformar a coluna em INT
df1.drop(df1[(df1.CO_ANO == '2°07') | (df1.CO_ANO == '2°06')].index, inplace=True)

In [ ]:
# dropadas as colunas problematicas, é possível converter a coluna e possibilitar sua filtragem 
# por meio de uma comparação matemática simples
df1['CO_ANO'] = df1['CO_ANO'].astype(int)

In [ ]:
df1.dtypes

CO_ANO         int64
CO_MES         int64
SH4            int64
CO_PAIS        int64
SG_UF_MUN     object
CO_MUN         int64
KG_LIQUIDO     int64
VL_FOB         int64
dtype: object

In [ ]:
# Realizando um filtro dos ultimos 10 anos (2012 à 2022)
# Além disso, o df1 será sobrescrito para diminuir o gasto de recursos
filtro_ano  = df1['CO_ANO'] >= 2012
df1 = df1[filtro_ano]
df1

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB
12649165,2012,8,8501,493,BA,2927408,65,2193
12649166,2012,11,8205,160,SC,4208203,124777,247680
12649167,2012,1,6115,160,RS,4314902,2512,37327
12649168,2012,12,9114,249,AM,1302603,47,39584
12649169,2012,7,8481,361,SP,3409502,155,58987
...,...,...,...,...,...,...,...,...
28456808,2022,4,3504,160,SP,3408405,75,6000
28456809,2022,4,3916,23,SP,3418404,1,49
28456810,2022,10,8537,386,RS,4304309,5,848
28456811,2022,8,9031,767,SP,3415004,1,1918


In [ ]:
# conferência simples, que permite visualizar o tamanho do DataFrame
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15807648 entries, 12649165 to 28456812
Data columns (total 8 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   CO_ANO      int64 
 1   CO_MES      int64 
 2   SH4         int64 
 3   CO_PAIS     int64 
 4   SG_UF_MUN   object
 5   CO_MUN      int64 
 6   KG_LIQUIDO  int64 
 7   VL_FOB      int64 
dtypes: int64(7), object(1)
memory usage: 1.1+ GB


### Upload da base de importação com base nos municípios, com 10 anos selecionados

In [ ]:
# SALVAMENTO LOCAL DO DF COMO CSV
df1.to_csv("IMP_2012-2022 (MUN).csv", index=False)

In [ ]:
# NOVAMENTE SERÁ FEITO O ACESSO À BUCKET PARA O UPLOAD DESSA BASE 
client = storage.Client()
bucket = client.get_bucket('bc26-datasets-comex')

In [ ]:
# UPLOAD DO CSV, PARA A BUCKET DO GOOGLE CLOUD
blob = bucket.blob('IMP_2012-2022 (MUN).csv') 
blob.upload_from_filename('IMP_2012-2022 (MUN).csv')

Foram feitos alguns testes e o pandas consegue transformar esse CSV com os 10 anos de importação em um DataFrame, levando aproximadamente 3 minutos, dada essa grande quantidade de tempo e considerando que o Colab as vezes falha, exigindo uma nova criação do DF, a base de dados será salva também no formato "parquet" para buscar eficiência

In [ ]:
# criação do caminho dentro da bucket (que ja está acessada) para salvar o parquet
path = 'gs://bc26-datasets-comex/IMP_2012-2022 (MUN).parquet'

In [ ]:
# SALVAMENTO DO ARQUIVO NO FORMATO PARQUET DENTRO DA BUCKET
df1.to_parquet(path, index=False, partition_cols=["CO_ANO"])

In [ ]:
# COMPROVAÇÃO DA EFICIÊNCIA DO FORMATO PARQUET:
# O DF QUE LEVAVA APROXIMADAMENTE 3 MINUTOS PARA SER MONTADO A PARTIR DO CSV
# AGORA É MONTADO PELO PANDAS EM APROXIMADAMENTE 10 SEGUNDOS 
# (ganho de eficiência nada negligenciável)
df1 = pd.read_parquet(path)

In [ ]:
df1

,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,CO_ANO
0,8,8501,493,BA,2927408,65,2193,2012
1,11,8205,160,SC,4208203,124777,247680,2012
2,1,6115,160,RS,4314902,2512,37327,2012
3,12,9114,249,AM,1302603,47,39584,2012
4,7,8481,361,SP,3409502,155,58987,2012
...,...,...,...,...,...,...,...,...
15807643,4,8501,23,SP,3445803,6095,114744,2022
15807644,9,4016,245,SP,3405708,89,2670,2022
15807645,4,4809,249,SC,4208450,137,10890,2022
15807646,3,4823,72,SC,4208203,27,738,2022


## Dataset de Importação com baseada nos produtos

In [ ]:
# CONFIGURAÇÃO DA CHAVE DE SEGURANÇA PARA CONEXÃO COM A GCP
serviceAccount = '/content/chave_bucket_JSON.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# ACESSO DA BUCKET
client = storage.Client()

# VARIÁVEL PARA RECEBER O NOME DA BUCKET 
bucket = client.get_bucket('bc26-datasets-comex')

# SELEÇÃO DO ARQUIVO NA BUCKET
bucket.blob('IMP_COMPLETA (NCM).csv')

# VARIÁVEL PARA RECEBER O CAMINHO DO ARQUIVO
path = 'gs://bc26-datasets-comex/IMP_COMPLETA (NCM).csv'

In [ ]:
#CRIAÇÃO DO DF A PARTIR DO CSV (BASE DE DADOS DE IMPORTAÇÃO, BASEADA NOS PRODUTOS)
df1 = pd.read_csv(path, sep=";", encoding='ISO-8859-1')

In [ ]:
df1

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO
0,1997,9,29251990,10,87,SP,1,817800,200,200,4888,180,24
1,1997,8,85049010,10,399,SP,4,817600,4891,84,30116,409,93
2,1997,8,54023300,10,63,SP,7,1010900,143834,143834,402427,10121,266
3,1997,3,38121000,10,249,SP,1,817800,9434,9172,61190,3011,163
4,1997,10,73209000,10,399,SP,4,817600,222510,49,3638,456,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38090163,2022,5,29211923,10,249,CE,1,817800,29810,29810,54734,3625,48
38090164,2022,5,29329999,10,160,GO,1,817800,225,225,2634,133,0
38090165,2022,4,35069110,10,249,PR,1,817800,15946,15946,138883,1121,421
38090166,2022,10,90184100,11,160,SP,4,817600,44,12,849,801,0


In [ ]:
# Realizando um filtro dos ultimos 10 anos (2012 à 2022)
# Além disso, o df1 será sobrescrito para diminuir o gasto de recursos
filtro_ano  = df1['CO_ANO'] >= 2012
df1 = df1[filtro_ano]
df1

TypeError: ignored

### Upload da base de importação com base nos produtos, com 10 anos selecionados

Dado o ganho de performance, anteriormente verificado, o upload do dataset (com os 10 anos selecionados com base nos produtos) será feito na bucket no formato "parquet"

In [ ]:
# NOVAMENTE SERÁ FEITO O ACESSO À BUCKET PARA O UPLOAD DESSA BASE 
client = storage.Client()
bucket = client.get_bucket('bc26-datasets-comex')

In [ ]:
# criação do caminho dentro da bucket (que ja está acessada) para salvar o parquet
path = 'gs://bc26-datasets-comex/IMP_2012-2022 (NCM).parquet'

In [ ]:
# SALVAMENTO DO ARQUIVO NO FORMATO PARQUET DENTRO DA BUCKET
df1.to_parquet(path, index=False, partition_cols=["CO_ANO"])

In [ ]:
df1

,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO,CO_ANO
0,10,85441990,10,399,SP,4,817700,0,0,134,24,0,2012
1,10,85115010,11,149,SP,4,817600,1,1,36833,21,11,2012
2,3,90013000,11,365,SP,4,817600,6950,153,20755,992,8,2012
3,4,39269090,10,493,SP,4,817600,219,219,3180,3659,4,2012
4,3,58012100,10,160,SC,1,927800,21101,21101,55922,3372,0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20389857,8,85443000,10,820,AM,1,227600,641,641,24284,658,2,2022
20389858,11,85043199,11,351,SC,4,817700,4,9,508,350,0,2022
20389859,6,85129000,10,247,SP,1,817800,6,6,150,3,0,2022
20389860,7,85177900,10,538,RJ,4,717700,36,36,3380,1010,0,2022


In [ ]:
df1.SG_UF_NCM.value_counts()

SP    7992210
SC    2025429
PR    1933550
RJ    1628018
MG    1511526
RS    1304628
AM    1023594
ES     525376
BA     522472
GO     382783
PE     327222
CE     253448
RO     143005
PA     140249
AL     136830
MS     102037
DF      97210
PB      66151
RN      64255
MT      48781
SE      46798
MA      40284
AP      26241
TO      21871
PI      17596
RR       5980
AC       1919
EX        291
ND        108
Name: SG_UF_NCM, dtype: int64

# Tratamento da base referentes à importação baseada nos municípios

In [ ]:
# CONFIGURAÇÃO DA CHAVE DE SEGURANÇA PARA CONEXÃO COM A GCP
serviceAccount = '/content/chave_bucket_JSON.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# ACESSO NA BUCKET PARA O ACESSAR A BASE 
client = storage.Client()
bucket = client.get_bucket('bc26-datasets-comex')

In [ ]:
# criação do caminho dentro da bucket (que ja está acessada) para acessar o arquivo
path = 'gs://bc26-datasets-comex/IMP_2012-2022 (MUN).parquet'

In [ ]:
df1 = pd.read_parquet(path)

In [ ]:
df1

,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,CO_ANO
0,8,8501,493,BA,2927408,65,2193,2012
1,11,8205,160,SC,4208203,124777,247680,2012
2,1,6115,160,RS,4314902,2512,37327,2012
3,12,9114,249,AM,1302603,47,39584,2012
4,7,8481,361,SP,3409502,155,58987,2012
...,...,...,...,...,...,...,...,...
15807643,4,8501,23,SP,3445803,6095,114744,2022
15807644,9,4016,245,SP,3405708,89,2670,2022
15807645,4,4809,249,SC,4208450,137,10890,2022
15807646,3,4823,72,SC,4208203,27,738,2022


## Remoção de possíveis espaços vazios antes ou depois dos dados

In [ ]:
# o código abaixo itera pelas colunas removendo espaços antes e depois de cada entrada
# caso haja incompatibilidade e uma coluna com a função "strip()", não haverá erro, entrará na exceção e seguirá para a próxima coluna
for col in df1.columns:
  try:
    df1[col] = df1[col].apply(lambda x: x.strip())
  except:
    pass

## Exploração das colunas que trazem códigos

O dataset já possuia muito poucas inconsistências e foi pré-tratado nos passos anteriores, agora o dataframe vai ser conectado (com uso das funções "join" e "merge") com as tabelas auxiliares e trazer para o df principal o significado dos códigos: "SH4" e "CO_MUN"

### código "SH4"

In [ ]:
# primeiro puxar em "df_referência" a página com a tabela auxiliar que faz referência ao código SH4
df_referencia = pd.read_excel('gs://bc26-datasets-comex/TABELAS_AUXILIARES.xlsx', sheet_name=1)

In [ ]:
df_referencia

,CO_NCM,NO_NCM_POR,CO_SH6,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,CO_SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING
0,1011010,Cavalos reprodutores de raça pura,10110,"Animais vivos das espécies cavalar, asinina e ...","Caballos y asnos, reproductores de raza pura",Pure-bred breeding horses and asses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",10110,"Animais vivos das espécies cavalar, asinina e ...","Caballos y asnos, reproductores de raza pura",Pure-bred breeding horses and asses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
2,1011100,"Cavalos reprodutores, de raça pura",10111,"Cavalos reprodutores, de raça pura",Caballos reproductores de raza pura,Pure-bred breeding horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
3,1011900,"Outros cavalos, vivos",10119,"Outros cavalos, vivos","Demás caballos, vivos",Other live horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
4,1012000,Asininos e muares vivos,10120,Asininos e muares vivos,"Asnos, mulos y burdéganos, vivos","Asses, mules and hinnies (live)",101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13593,99997103,Joalheria de ouro do capitulo 71 da NCM,999971,"Pedras, em bruto ou trabalhadas, do Capítulo 71","Piedras, en bruto o no, del capítulo 71","Stones, raw or worked, from Chapter 71",9999,Consumo de bordo (exceto combustíveis e lubrif...,Consumo a bordo de buques y aeronaves (exc.com...,Consumption on board ships and aircraft (excep...,99,Transações especiais,Transaciones especiales,Special operations,XXII,Transações especiais,Operaciones especiales,Special transactions
13594,99997104,Outros artigos do capitulo 71 da NCM,999971,"Pedras, em bruto ou trabalhadas, do Capítulo 71","Piedras, en bruto o no, del capítulo 71","Stones, raw or worked, from Chapter 71",9999,Consumo de bordo (exceto combustíveis e lubrif...,Consumo a bordo de buques y aeronaves (exc.com...,Consumption on board ships and aircraft (excep...,99,Transações especiais,Transaciones especiales,Special operations,XXII,Transações especiais,Operaciones especiales,Special transactions
13595,99999900,Qualquer outra mercadoria sem cobertura cambial,999999,Qualquer outra mercadoria sem cobertura cambia...,Todos otros bienes sin cobertura de cambios y...,Any other goods without exchange cover; reexport,9999,Consumo de bordo (exceto combustíveis e lubrif...,Consumo a bordo de buques y aeronaves (exc.com...,Consumption on board ships and aircraft (excep...,99,Transações especiais,Transaciones especiales,Special operations,XXII,Transações especiais,Operaciones especiales,Special transactions
13596,99999951,Devolução antes da DI (expor

In [ ]:
# renomear coluna "CO_SH4" para ficar igual ao df1: "SH4"
df_referencia.rename(columns={'CO_SH4':'SH4'},inplace=True)

In [ ]:
#fazer o join das colunas trazendo apenas a coluna que "explica" os códigos "SH4" no dataset principal
df1 = df1.join(df_referencia[["NO_SH4_POR"]], on='SH4')

In [ ]:
# conferindo que nenhum valor na nova coluna ficou nulo
df1.NO_SH4_POR.isnull().sum()

0

### código "CO_MUN"

In [ ]:
# os passos anteriores serão repitidos trazendo as referências para o código "CO_MUN"
df_referencia = pd.read_excel('gs://bc26-datasets-comex/TABELAS_AUXILIARES.xlsx', sheet_name=13)

In [ ]:
df_referencia

,CO_MUN_GEO,NO_MUN,NO_MUN_MIN,SG_UF
0,1501758,BREJO GRANDE DO ARAGUAIA,Brejo Grande do Araguaia,PA
1,1501782,BREU BRANCO,Breu Branco,PA
2,1501808,BREVES,Breves,PA
3,1501907,BUJARU,Bujaru,PA
4,1501956,CACHOEIRA DO PIRIA,Cachoeira do Piriá,PA
...,...,...,...,...
5565,4322350,UNIAO DA SERRA,União da Serra,RS
5566,4322376,UNISTALDA,Unistalda,RS
5567,4322400,URUGUAIANA,Uruguaiana,RS
5568,4322509,VACARIA,Vacaria,RS


In [ ]:
# renomear coluna "CO_MUN_GEO" para ficar igual ao df1: "CO_MUN"
df_referencia.rename(columns={'CO_MUN_GEO':'CO_MUN'},inplace=True)

In [ ]:
# aqui também será feito um merge das tabelas, posteriormente serão removidas as redundantes
df1 = pd.merge(df1, df_referencia, on='CO_MUN')

In [ ]:
# Também serão dropadas as colunas redundantes
df1.drop(['SG_UF','NO_MUN'], axis=1, inplace=True)

### Código "CO_MES"

In [ ]:
 #Renomeando manualmente os valores da coluna "CO_MES"
df1['CO_MES']=df1['CO_MES'].replace({1:'janeiro',2:'fevereiro', 3:'março', 4:'abril',5:'maio', 6:'junho',7:'julho', 8:'agosto', 9:'setembro', 10:'outubro', 11:'novembro', 12 :'dezembro'})

In [ ]:
df1.CO_MES.value_counts()

outubro      1383778
agosto       1379383
novembro     1359028
julho        1357400
setembro     1345296
março        1343465
maio         1323317
junho        1307381
janeiro      1303856
abril        1287864
fevereiro    1254355
dezembro     1162525
Name: CO_MES, dtype: int64

## Renomeação das colunas

In [ ]:
df1.dtypes

CO_MES          object
SH4              int64
CO_PAIS          int64
SG_UF_MUN       object
CO_MUN           int64
KG_LIQUIDO       int64
VL_FOB           int64
CO_ANO        category
NO_SH4_POR      object
NO_MUN_MIN      object
dtype: object

In [ ]:
df1.rename(columns={'CO_MES':'mes',
                   'CO_PAIS':'cod_pais',
                   'SG_UF_MUN':'estado',
                   'CO_MUN':'cod_municipio',
                   'KG_LIQUIDO':'peso_liquido',
                   'VL_FOB':'valor_dolar',
                   'CO_ANO':'ano',
                   'NO_SH4_POR':'descricoes__sistema_harmonizado',
                   'CO_BLOCO':'cod_bloco',
                   'NO_BLOCO':'continente',
                   'NO_MUN_MIN':'municipio',
                  }, inplace=True)

In [ ]:
df1

,mes,SH4,cod_pais,estado,cod_municipio,peso_liquido,valor_dolar,ano,descricoes__sistema_harmonizado,municipio
0,agosto,8501,493,BA,2927408,65,2193,2012,"Camisolas interiores, cuecas, ceroulas, camisa...",Salvador
1,junho,5703,160,BA,2927408,7750,43694,2012,"Insecticidas, rodenticidas, fungicidas, herbic...",Salvador
2,abril,4016,249,BA,2927408,577,21586,2012,"Compostos heterocíclicos, exclusivamente de he...",Salvador
3,novembro,7009,603,BA,2927408,202,6259,2012,"Obras de carpintaria para construções, incluíd...",Salvador
4,janeiro,8708,249,BA,2927408,49737,657074,2012,"Chapéus e outros artefactos de uso semelhante,...",Salvador
...,...,...,...,...,...,...,...,...,...,...
15807643,agosto,8465,160,RS,4323358,2565,12977,2022,"Fatos de saia-casaco, conjuntos, casacos, vest...",Vila Lângaro
15807644,maio,3901,249,SC,4214409,93500,140624,2022,"Compostos heterocíclicos, exclusivamente de he...",Rio das Antas
15807645,fevereiro,8541,160,RN,2405603,5394,27351,2022,Lenços de assoar e de bolso,Jardim de Piranhas
15807646,agosto,3004,63,RS,4314027,1,38651,2022,Ácidos monocarboxílicos acíclicos saturados e ...,Paraíso do Sul


## Busca por demais inconsistências

In [ ]:
# CHECAR SE HÁ LINHAS DUPLICADAS
df1.duplicated(keep=False).sum()

0

In [ ]:
# CHECAR SE HÁ VALORES NULOS
df1.isnull().sum()

mes                                0
SH4                                0
cod_pais                           0
estado                             0
cod_municipio                      0
peso_liquido                       0
valor_dolar                        0
ano                                0
descricoes__sistema_harmonizado    0
municipio                          0
dtype: int64

In [ ]:
# CHECAR O TIPO DOS DADOS
df1.dtypes

mes                                  object
SH4                                   int64
cod_pais                              int64
estado                               object
cod_municipio                         int64
peso_liquido                          int64
valor_dolar                           int64
ano                                category
descricoes__sistema_harmonizado      object
municipio                            object
dtype: object

## Upload da base tratada

### Upload para a bucket na GCP

In [ ]:
# CONFIGURAÇÃO DA CHAVE DE SEGURANÇA PARA CONEXÃO COM A GCP
serviceAccount = '/content/chave_bucket_JSON.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# ACESSO NA BUCKET PARA O ACESSAR A BASE 
client = storage.Client()
bucket = client.get_bucket('bc26-datasets-comex')

In [ ]:
# criação do caminho dentro da bucket (que ja está acessada) para salvar o arquivo
path = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (MUN).parquet'

In [ ]:
# SALVAMENTO DO ARQUIVO NO FORMATO PARQUET DENTRO DA BUCKET
df1.to_parquet(path, index=False, partition_cols=["ano"])

In [ ]:
# para voltar a baixar o parquet, caso seja necessário criar novamente o df1 com o arquivo tratado
df1 = pd.read_parquet(path)

In [ ]:
df1

,mes,mercadoria,cod_unidade,cod_pais,estado,cod_urf,qnt_estatistica,peso_liquido,valor_dolar,produto,unidade_medida,meio_transporte,ano
0,março,39095021,10,190,CE,317700,211,211,3600,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012
1,junho,39095021,10,245,RS,1017600,50,50,459,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012
2,março,39095021,10,23,RS,1017600,1600,1600,21300,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012
3,julho,39095021,10,245,RS,1017600,200,200,1422,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012
4,agosto,39095021,10,161,SP,817600,1700,1700,9880,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20389443,maio,42022900,11,249,SP,817600,64,4,4746,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022
20389444,fevereiro,42022900,11,87,SP,817600,7,330,197,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022
20389445,junho,42022900,11,249,RJ,717700,16,345,1489,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022
20389446,maio,42022900,11,105,SP,817600,1,0,28,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022


### Upload para BigQuery

In [ ]:
df_bq = df1.copy()

In [ ]:
df_bq

,mes,SH4,cod_pais,estado,cod_municipio,peso_liquido,valor_dolar,descricoes__sistema_harmonizado,municipio,ano
0,janeiro,6115,160,RS,4314902,2512,37327,"Polímeros de etileno, em formas primárias",Porto Alegre,2012
1,fevereiro,8481,23,RS,4314902,80,8444,"Fatos de saia-casaco, conjuntos, casacos, vest...",Porto Alegre,2012
2,dezembro,7009,160,RS,4314902,23700,20412,"Obras de carpintaria para construções, incluíd...",Porto Alegre,2012
3,novembro,6811,63,RS,4314902,23226,10707,"Peles com pelo artificiais, e suas obras",Porto Alegre,2012
4,setembro,8421,399,RS,4314902,17,7349,"Sobretudos, japonas, gabões, capas, anoraques,...",Porto Alegre,2012
...,...,...,...,...,...,...,...,...,...,...
15807643,setembro,8536,386,ES,3205010,2,499,"Soutiens, cintas, espartilhos, suspensórios, l...",Sooretama,2022
15807644,junho,7321,160,ES,3205010,2052,6129,"Outros papéis, cartões, pasta (ouate) de celul...",Sooretama,2022
15807645,junho,8416,23,ES,3205010,75,9889,"Outros acessórios de vestuário, confeccionados...",Sooretama,2022
15807646,maio,8421,767,ES,3205010,476,47083,"Sobretudos, japonas, gabões, capas, anoraques,...",Sooretama,2022


In [ ]:
df_bq['ano'] = df_bq['ano'].astype(int)

In [ ]:
df_bq.dtypes

mes                                object
SH4                                 int64
cod_pais                            int64
estado                             object
cod_municipio                       int64
peso_liquido                        int64
valor_dolar                         int64
descricoes__sistema_harmonizado    object
municipio                          object
ano                                 int64
dtype: object

In [ ]:
# Chave de acesso ao BigQuery com as seguintes roles: Big Query User e Big Query Data Editor
key_path = "/content/chave_bigquery_JSON.json"

# Variável credentials
credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "bc26-ed7-projeto-final"

In [ ]:
# Envio para BigQuerry (dataset = 'comercio-brasil / tabela = 'imp_2012-2022_tratada_mun')

pandas_gbq.to_gbq(df_bq, 'comercio_brasil.imp_2012-2022_tratada_mun', project_id='bc26-ed7-projeto-final', if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 2046.00it/s]


# Dataset baseado nos produtos

In [ ]:
# CONFIGURAÇÃO DA CHAVE DE SEGURANÇA PARA CONEXÃO COM A GCP
serviceAccount = '/content/chave_bucket_JSON.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# ACESSO NA BUCKET PARA O ACESSAR A BASE 
client = storage.Client()
bucket = client.get_bucket('bc26-datasets-comex')

In [ ]:
# criação do caminho dentro da bucket (que ja está acessada) para acessar o arquivo
path = 'gs://bc26-datasets-comex/IMP_2012-2022 (NCM).parquet'

In [ ]:
df1 = pd.read_parquet(path)

In [ ]:
df1

,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO,CO_ANO
0,10,85441990,10,399,SP,4,817700,0,0,134,24,0,2012
1,10,85115010,11,149,SP,4,817600,1,1,36833,21,11,2012
2,3,90013000,11,365,SP,4,817600,6950,153,20755,992,8,2012
3,4,39269090,10,493,SP,4,817600,219,219,3180,3659,4,2012
4,3,58012100,10,160,SC,1,927800,21101,21101,55922,3372,0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20389857,8,85443000,10,820,AM,1,227600,641,641,24284,658,2,2022
20389858,11,85043199,11,351,SC,4,817700,4,9,508,350,0,2022
20389859,6,85129000,10,247,SP,1,817800,6,6,150,3,0,2022
20389860,7,85177900,10,538,RJ,4,717700,36,36,3380,1010,0,2022


* Significados das colunas:

"
CO_ANO: Ano (1997 a 2021)

CO_MES: Código do mês (1:Janeiro a 12:Dezembro)

CO_NCM: Código da Nomenclatura Comum Mercosul - Utilizada para controle e identificação das mercadorias comercializadas no Brasil e nos outros países do Mercosul (cada NCM representa um produto diferente)

CO_UNID: Código da Unidade de Medida Estatística que é uma unidade de medida padrão para cada NCM, podendo ter valores como quilograma, metro, litro, pares, tonelada e outros.

CO_PAIS: Código do nome do país com a qual foi realizada a operação comercial (importação ou exportação)

SG_UF _NCM (sigla UF origem/destino da NCM): Código da Unidade Federativa (estado) de origem (exportação) ou destino (importação) da mercadoria.

CO_VIA: Código para identificação do meio de transporte utilizado (aéreo, marítimo, rodoviária, ferroviária e outros). Na exportação, é o método utilizado para o transporte de mercadorias entre o último local de embarque para o exterior. Na importação, configura-se através dos meios de acesso para os bens do primeiro ponto de entrada no território nacional.

CO_URF (Unidade da Receita Federal): Código da agência responsável pela execução dos procedimentos necessários para o desembaraço aduaneiro da mercadoria importada/exportada

QT_ESTAT: No detalhamento por NCM, cada produto tem sua unidade estatística. Grande parte dos produtos tem como unidade estatística o peso em quilogramas, mas existem outras: quilograma líquido, número (unidades), pares, dúzias, milheiro, tonelada. A tabela completa que relaciona cada NCM com sua unidade estatística pode ser encontrada em na tabela “NCM _ UNIDADE”. É importante ressaltar que não se deve somar quantidades estatísticas de NCMs que contenham unidades estatísticas diferentes. [2]

KG_LIQUIDO: Medida que expressa o peso líquido da mercadoria. Mesmo produtos com quantidades estatísticas diferentes do quilograma também possuem disponível a medida em quilograma, referindo-se ao peso líquido da mercadoria, ou seja, mercadoria desconsiderando embalagens, caixas ou quaisquer outros adicionais de transporte. Vale relembrar que essa informação, bem como as demais informadas nas operações de comércio exterior, é de livre preenchimento e de responsabilidade exclusiva dos operadores de comércio exterior. [2]

VL_FOB: O valor FOB indica o preço da mercadoria em dólares americanos (US$) sob o Incoterm FOB (Free on Board), modalidade na qual o vendedor é responsável por embarcar a mercadoria enquanto o comprador assume o pagamento do frete, seguros e demais custos pós embarque."

## Remoção de possíveis espaços vazios antes ou depois dos dados

In [ ]:
# o código, já usado anteriormente, removerá espaços brancos que sobrem nos dados, se houver
for col in df1.columns:
  try:
    df1[col] = df1[col].apply(lambda x: x.strip())
  except:
    pass

## Exploração das colunas que trazem códigos

No próximo passo, será feita a junção do df principal (df1) com tabelas de referência para trazer o significado de alguns códigos: 
"CO_NCM"	"CO_UNID"	"CO_VIA"

Nem todos os códigos precisam ser agregados nesse passo, visto que, posteriormente, em período de análise a agregação dos significados dos códigos pode ser feita por SQL

### código "CO_NCM"

In [ ]:
# primeiro puxar em "df_referência" a página com a tabela auxiliar que faz referência ao código CO_NCM
df_referencia = pd.read_excel('gs://bc26-datasets-comex/TABELAS_AUXILIARES.xlsx', sheet_name=7)

In [ ]:
df_referencia

,CO_NCM,NO_NCM_POR,CO_FAT_AGREG,NO_FAT_AGREG,NO_FAT_AGREG_GP
0,1011010,Cavalos reprodutores de raça pura,1,PRODUTOS BASICOS,PRODUTOS BASICOS
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",1,PRODUTOS BASICOS,PRODUTOS BASICOS
2,1011100,"Cavalos reprodutores, de raça pura",1,PRODUTOS BASICOS,PRODUTOS BASICOS
3,1011900,"Outros cavalos, vivos",1,PRODUTOS BASICOS,PRODUTOS BASICOS
4,1012000,Asininos e muares vivos,1,PRODUTOS BASICOS,PRODUTOS BASICOS
...,...,...,...,...,...
13578,99997103,Joalheria de ouro do capitulo 71 da NCM,4,TRANSACOES ESPECIAIS,OPERACOES ESPECIAIS
13579,99997104,Outros artigos do capitulo 71 da NCM,4,TRANSACOES ESPECIAIS,OPERACOES ESPECIAIS
13580,99999900,Qualquer outra mercadoria sem cobertura cambial,4,TRANSACOES ESPECIAIS,OPERACOES ESPECIAIS
13581,99999951,Devolução antes da DI (exportação definitiva),6,REEXPORTACAO,OPERACOES ESPECIAIS


In [ ]:
#fazer o merge trazendo as colunas do df_referencia
df1 = pd.merge(df1, df_referencia, on='CO_NCM')

In [ ]:
df1

,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO,CO_ANO,NO_NCM_POR,CO_FAT_AGREG,NO_FAT_AGREG,NO_FAT_AGREG_GP
0,10,85441990,10,399,SP,4,817700,0,0,134,24,0,2012,"Outros fios para bobinar, isolados para usos e...",3,PRODUTOS MANUFATURADOS,PRODUTOS INDUSTRIALIZADOS
1,7,85441990,10,249,RJ,4,717700,23,23,3564,104,6,2012,"Outros fios para bobinar, isolados para usos e...",3,PRODUTOS MANUFATURADOS,PRODUTOS INDUSTRIALIZADOS
2,2,85441990,10,351,SP,1,817800,3232,3232,24254,683,115,2012,"Outros fios para bobinar, isolados para usos e...",3,PRODUTOS MANUFATURADOS,PRODUTOS INDUSTRIALIZADOS
3,7,85441990,10,399,SP,4,817600,7,7,1670,70,6,2012,"Outros fios para bobinar, isolados para usos e...",3,PRODUTOS MANUFATURADOS,PRODUTOS INDUSTRIALIZADOS
4,10,85441990,10,23,SP,4,817600,67,67,4189,299,5,2012,"Outros fios para bobinar, isolados para usos e...",3,PRODUTOS MANUFATURADOS,PRODUTOS INDUSTRIALIZADOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20389443,9,3054990,10,399,SP,1,817800,9160,9160,93619,10200,185,2022,"Outros peixes defumados, mesmo em filés",1,PRODUTOS BASICOS,PRODUTOS BASICOS
20389444,8,3054990,10,399,SP,1,817800,9160,9160,97632,10200,193,2022,"Outros peixes defumados, mesmo em filés",1,PRODUTOS BASICOS,PRODUTOS BASICOS
20389445,11,29394510,10,361,SP,4,817600,0,0,2800,199,2,2022,Levometanfetamina e seus sais,3,PRODUTOS MANUFATURADOS,PRODUTOS INDUSTRIALIZADOS
20389446,7,89033300,11,386,SC,1,927800,1,130000,7220847,376086,15043,2022,"Barcos a motor, exceto os infláveis, não equip...",3,PRODUTOS MANUFATURADOS,PRODUTOS INDUSTRIALIZADOS


In [ ]:
# ALGUMAS COLUNAS SERÃO REMOVIDAS CONSIDERANDO QUE AINDA SERÁ POSSÍVEL JUNTÁ-LAS PARA ANÁLISE, POSTERIORMENTE.
df1.drop(['CO_FAT_AGREG',	'NO_FAT_AGREG', 'NO_FAT_AGREG_GP'], axis=1, inplace=True)

### código "CO_UNID"

In [ ]:
# primeiro puxar em "df_referência" a página com a tabela auxiliar que faz referência ao código CO_NCM
df_referencia = pd.read_excel('gs://bc26-datasets-comex/TABELAS_AUXILIARES.xlsx', sheet_name=6)

In [ ]:
df_referencia

,CO_NCM,NO_NCM_POR,CO_UNID,NO_UNID,SG_UNID
0,1011010,Cavalos reprodutores de raça pura,11,NUMERO (UNIDADE),UNID.
1,1011090,"Animais da espécie asinina/muar, reprodutores,...",11,NUMERO (UNIDADE),UNID.
2,1011100,"Cavalos reprodutores, de raça pura",11,NUMERO (UNIDADE),UNID.
3,1011900,"Outros cavalos, vivos",11,NUMERO (UNIDADE),UNID.
4,1012000,Asininos e muares vivos,11,NUMERO (UNIDADE),UNID.
...,...,...,...,...,...
13593,99997103,Joalheria de ouro do capitulo 71 da NCM,10,QUILOGRAMA LIQUIDO,KGL
13594,99997104,Outros artigos do capitulo 71 da NCM,10,QUILOGRAMA LIQUIDO,KGL
13595,99999900,Qualquer outra mercadoria sem cobertura cambial,10,QUILOGRAMA LIQUIDO,KGL
13596,99999951,Devolução antes da DI (exportação definitiva),10,QUILOGRAMA LIQUIDO,KGL


In [ ]:
#fazer o join das colunas trazendo apenas a coluna que "explica" os códigos "CO_UNID" no dataset principal
df1 = df1.join(df_referencia[["NO_UNID"]], on='CO_UNID')

In [ ]:
# conferindo que nenhum valor na nova coluna ficou nulo
df1.NO_UNID.isnull().sum()

0

### código "CO_VIA"

In [ ]:
# os passos anteriores serão repitidos trazendo as referências para o código "CO_VIA"
df_referencia = pd.read_excel('gs://bc26-datasets-comex/TABELAS_AUXILIARES.xlsx', sheet_name=14)

In [ ]:
df_referencia

,CO_VIA,NO_VIA
0,0,VIA NAO DECLARADA
1,1,MARITIMA
2,2,FLUVIAL
3,3,LACUSTRE
4,4,AEREA
5,5,POSTAL
6,6,FERROVIARIA
7,7,RODOVIARIA
8,8,CONDUTO/REDE DE TRANSMISSAO
9,9,MEIOS PROPRIOS


In [ ]:
# merge das tabelas
df1 = pd.merge(df1, df_referencia, on='CO_VIA')

In [ ]:
# visto que o código acima fazia referência apenas à uma coluna que agora faz parte do dataframe, a coluna CO_VIA será dropada
df1.drop(['CO_VIA'], axis=1, inplace=True)

### Código "CO_MES"

In [ ]:
 # Novamente a "CO_MES" pode ser "manualmente" renomeada
df1['CO_MES']=df1['CO_MES'].replace({1:'janeiro',2:'fevereiro', 3:'março', 4:'abril',5:'maio', 6:'junho',7:'julho', 8:'agosto', 9:'setembro', 10:'outubro', 11:'novembro', 12 :'dezembro'})

In [ ]:
df1.CO_MES.value_counts()

outubro      1789625
agosto       1787693
julho        1754204
novembro     1751369
setembro     1739396
março        1731781
maio         1704531
junho        1689340
janeiro      1674043
abril        1660352
fevereiro    1611606
dezembro     1495508
Name: CO_MES, dtype: int64

## Renomeação das colunas

In [ ]:
df1.rename(columns={'CO_MES':'mes',
                    'CO_NCM':'mercadoria',
                    'CO_UNID':'cod_unidade',
                    'CO_PAIS':'cod_pais',
                    'SG_UF_NCM':'estado',
                    'CO_URF':'cod_urf',
                    'QT_ESTAT':'qnt_estatistica',
                    'KG_LIQUIDO':'peso_liquido',
                    'VL_FOB':'valor_dolar',
                    'CO_ANO':'ano',
                    'NO_NCM_POR':'produto',
                    'NO_UNID':'unidade_medida',
                    'NO_VIA':'meio_transporte',
                   }, inplace=True)

In [ ]:
df1

,mes,mercadoria,cod_unidade,cod_pais,estado,cod_urf,qnt_estatistica,peso_liquido,valor_dolar,ano,produto,unidade_medida,meio_transporte
0,outubro,85441990,10,399,SP,817700,0,0,134,2012,"Outros fios para bobinar, isolados para usos e...",NUMERO (UNIDADE),AEREA
1,julho,85441990,10,249,RJ,717700,23,23,3564,2012,"Outros fios para bobinar, isolados para usos e...",NUMERO (UNIDADE),AEREA
2,julho,85441990,10,399,SP,817600,7,7,1670,2012,"Outros fios para bobinar, isolados para usos e...",NUMERO (UNIDADE),AEREA
3,outubro,85441990,10,23,SP,817600,67,67,4189,2012,"Outros fios para bobinar, isolados para usos e...",NUMERO (UNIDADE),AEREA
4,outubro,85441990,10,249,SP,817700,29,29,3190,2012,"Outros fios para bobinar, isolados para usos e...",NUMERO (UNIDADE),AEREA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20389443,fevereiro,27160000,18,845,GO,1010351,25080,0,4907822,2022,Energia elétrica,NUMERO (UNIDADE),CONDUTO/REDE DE TRANSMISSAO
20389444,março,27160000,18,845,RJ,1010351,14774,1,2812100,2022,Energia elétrica,NUMERO (UNIDADE),CONDUTO/REDE DE TRANSMISSAO
20389445,agosto,27160000,18,586,RJ,9999999,0,0,107733020,2022,Energia elétrica,NUMERO (UNIDADE),CONDUTO/REDE DE TRANSMISSAO
20389446,fevereiro,27160000,18,63,GO,1017500,26284,0,5361516,2022,Energia elétrica,NUMERO (UNIDADE),CONDUTO/REDE DE TRANSMISSAO


## Dropagem de colunas que não serão utilizadas

In [ ]:
# OS DADOS TRAZIDOS NAS COLUNAS ABAIXO FORAM CONSIDERADOS IMPRECISOS OU MENOS VALIOSOS PARA AS ANÁLISES POSTERIORES.
# PORTANTO SERÃO DROPADAS AS COLUNAS
df1.drop(['VL_FRETE', 'VL_SEGURO'], axis=1, inplace=True)

## Busca por demais inconsistências

In [ ]:
# CHECAR SE HÁ LINHAS DUPLICADAS
df1.duplicated(keep=False).sum()

0

In [ ]:
# CHECAR SE HÁ VALORES NULOS
df1.isnull().sum()

mes                0
mercadoria         0
cod_unidade        0
cod_pais           0
estado             0
cod_urf            0
qnt_estatistica    0
peso_liquido       0
valor_dolar        0
ano                0
produto            0
unidade_medida     0
meio_transporte    0
dtype: int64

In [ ]:
# CHECAR O TIPO DOS DADOS
df1.dtypes

mes                  object
mercadoria            int64
cod_unidade           int64
cod_pais              int64
estado               object
cod_urf               int64
qnt_estatistica       int64
peso_liquido          int64
valor_dolar           int64
ano                category
produto              object
unidade_medida       object
meio_transporte      object
dtype: object

## Upload da base tratada

### Upload para a GCP

In [ ]:
# CONFIGURAÇÃO DA CHAVE DE SEGURANÇA PARA CONEXÃO COM A GCP
serviceAccount = '/content/chave_bucket_JSON.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# ACESSO NA BUCKET PARA O ACESSAR A BASE 
client = storage.Client()
bucket = client.get_bucket('bc26-datasets-comex')

In [ ]:
# criação do caminho dentro da bucket (que ja está acessada) para salvar o arquivo
path = 'gs://bc26-datasets-comex/comex-tratado/IMP_2012-2022_tratada (NCM).parquet'

In [ ]:
# SALVAMENTO DO ARQUIVO NO FORMATO PARQUET DENTRO DA BUCKET
df1.to_parquet(path, index=False, partition_cols=["ano"])

### Correções pontuais

In [ ]:
# para a baixar o parquet com o arquivo tratado
df1 = pd.read_parquet(path)

In [ ]:
#alterando o tipo 'category', nativo ao 'parquet', dessa vez para aplicar um filtro, na sequência
df1['ano'] = df1['ano'].astype(int)

In [ ]:
# corrigindo equivoco na filtragem (últimos 10 anos)
filtro_ano  = df1['ano'] >= 2013
df1 = df1[filtro_ano]
df1

Criação de coluna data, no formato datetime do pandas

In [ ]:
# criando uma nova coluna com o número do mês, em string para guardar o zero
df1['n_mes']=df2['mes'].replace({'janeiro':'01','fevereiro':'02', 'março':'03', 'abril':'04','maio':'05', 'junho':'06','julho':'07', 'agosto':'08', 'setembro':'09', 'outubro':'10', 'novembro':'11', 'dezembro':'12'})

In [ ]:
# criação de coluna "dia", com o valor '01' visto que a base não traz o dia
df1['dia'] = '01'

In [ ]:
#função para criar a coluna data, com o valor string, com a concatenação das colunas
# ano,n_mes e dia, os dados da coluna serão strings assim 20220901
df1["data"] = df1.apply(lambda row: str(row["ano"]) + str(row["n_mes"]) + str(row["dia"]), axis=1)

In [ ]:
#transformando os valores da coluna 'data' em datetime
df1['data'] = pd.to_datetime(df1['data'], format='%Y-%m-%d')

In [ ]:
df1

,mes,mercadoria,cod_unidade,cod_pais,estado,cod_urf,qnt_estatistica,peso_liquido,valor_dolar,produto,unidade_medida,meio_transporte,ano,n_mes,dia,data
1829060,julho,39095021,10,628,SP,817600,50,50,746,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,07,01,2013-07-01
1829061,agosto,39095021,10,245,RS,1017600,225,225,1825,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,08,01,2013-08-01
1829062,julho,39095021,10,161,RS,1017600,10,10,130,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,07,01,2013-07-01
1829063,março,39095021,10,386,MG,615100,18,18,4320,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,03,01,2013-03-01
1829064,abril,39095021,10,23,SC,817700,200,200,742,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,04,01,2013-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20389443,maio,42022900,11,249,SP,817600,64,4,4746,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,05,01,2022-05-01
20389444,fevereiro,42022900,11,87,SP,817600,7,330,197,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,02,01,2022-02-01
20389445,junho,42022900,11,249,RJ,717700,16,345,1489,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,06,01,2022-06-01
20389446,maio,42022900,11,105,SP,817600,1,0,28,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,05,01,2022-05-01


In [ ]:
#dropagem das colunas que não terão mais interesse
df1.drop(['n_mes'], axis=1, inplace=True)

In [ ]:
df1.drop(['dia'], axis=1, inplace=True)

In [ ]:
df1

,mes,mercadoria,cod_unidade,cod_pais,estado,cod_urf,qnt_estatistica,peso_liquido,valor_dolar,produto,unidade_medida,meio_transporte,ano,data
1829060,julho,39095021,10,628,SP,817600,50,50,746,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-07-01
1829061,agosto,39095021,10,245,RS,1017600,225,225,1825,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-08-01
1829062,julho,39095021,10,161,RS,1017600,10,10,130,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-07-01
1829063,março,39095021,10,386,MG,615100,18,18,4320,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-03-01
1829064,abril,39095021,10,23,SC,817700,200,200,742,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20389443,maio,42022900,11,249,SP,817600,64,4,4746,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,2022-05-01
20389444,fevereiro,42022900,11,87,SP,817600,7,330,197,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,2022-02-01
20389445,junho,42022900,11,249,RJ,717700,16,345,1489,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,2022-06-01
20389446,maio,42022900,11,105,SP,817600,1,0,28,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,2022-05-01


### Upload para BigQuery

In [ ]:
df_bq = df1.copy()

In [ ]:
df_bq

,mes,mercadoria,cod_unidade,cod_pais,estado,cod_urf,qnt_estatistica,peso_liquido,valor_dolar,produto,unidade_medida,meio_transporte,ano,data
1829060,julho,39095021,10,628,SP,817600,50,50,746,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-07-01
1829061,agosto,39095021,10,245,RS,1017600,225,225,1825,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-08-01
1829062,julho,39095021,10,161,RS,1017600,10,10,130,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-07-01
1829063,março,39095021,10,386,MG,615100,18,18,4320,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-03-01
1829064,abril,39095021,10,23,SC,817700,200,200,742,"Poliuretanos hidroxilados, com propriedades ad...",NUMERO (UNIDADE),AEREA,2013,2013-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20389443,maio,42022900,11,249,SP,817600,64,4,4746,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,2022-05-01
20389444,fevereiro,42022900,11,87,SP,817600,7,330,197,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,2022-02-01
20389445,junho,42022900,11,249,RJ,717700,16,345,1489,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,2022-06-01
20389446,maio,42022900,11,105,SP,817600,1,0,28,Bolsas de outras matérias,NUMERO (UNIDADE),ENTRADA/SAIDA FICTA,2022,2022-05-01


In [ ]:
# Chave de acesso ao BigQuery com as seguintes roles: Big Query User e Big Query Data Editor
key_path= "/content/chave_bigquery_JSON.json"

# Variável credentials
credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "bc26-ed7-projeto-final"

In [ ]:
# Envio para BigQuerry (dataset = 'comercio-brasil / tabela = 'imp_2013-2022_tratada_ncm')

pandas_gbq.to_gbq(df_bq, 'comercio_brasil.imp_2013-2022_tratada_ncm', project_id='bc26-ed7-projeto-final', if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 6721.64it/s]
